<a href="https://colab.research.google.com/github/IvanKatorgin/Miscellaneous/blob/main/%D0%9A%D0%BE%D0%BF%D0%B8%D1%8F_%D0%B1%D0%BB%D0%BE%D0%BA%D0%BD%D0%BE%D1%82%D0%B0_%22%D0%A8%D0%B0%D0%B1%D0%BB%D0%BE%D0%BD_%D0%94%D0%97_ML_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Предсказываем стоимость мед страховки



## Загужаем необходимые библиотеки

In [ ]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

## Загружаем данные

Собраны данные:
- age: возраст
- sex: пол
- bmi: индекс массы тела
- children: количество детей, охваченных медицинским страхованием / количество иждивенцев
- smoker: курение
- region: регион (northeast, southeast, southwest, northwest).
- charges: индивидуальные медицинские расходы (его и хотим предсказать)

In [ ]:
# Загрузите данные из файла insurance.csv в переменную df
df=pd.read_csv('insurance.csv',',')

<ipython-input-5-66187b4645a7>:2: FutureWarning: In a future version of pandas all arguments of read_csv except for the argument 'filepath_or_buffer' will be keyword-only.
  df=pd.read_csv('insurance.csv',',')


In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


## Смотрим статистику, что нет пропусков и отсуствующих значений

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1338 entries, 0 to 1337
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   age       1338 non-null   int64  
 1   sex       1338 non-null   object 
 2   bmi       1338 non-null   float64
 3   children  1338 non-null   int64  
 4   smoker    1338 non-null   object 
 5   region    1338 non-null   object 
 6   charges   1338 non-null   float64
dtypes: float64(2), int64(2), object(3)
memory usage: 73.3+ KB


In [ ]:
df.describe()

,age,bmi,children,charges
count,1338.000000,1338.000000,1338.000000,1338.000000
mean,39.207025,30.663397,1.094918,13270.422265
std,14.049960,6.098187,1.205493,12110.011237
min,18.000000,15.960000,0.000000,1121.873900
25%,27.000000,26.296250,0.000000,4740.287150
50%,39.000000,30.400000,1.000000,9382.033000
75%,51.000000,34.693750,2.000000,16639.912515
max,64.000000,53.130000,5.000000,63770.428010


## Преобразуем строковые данные

In [ ]:
# Заменяем пол и курение на числа
df['sex']=df['sex'].map({'male':1, 'female':0})
df['smoker']=df['smoker'].map({'yes':1,'no':0})

In [ ]:
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,southwest,16884.92400
1,18,1,33.770,1,0,southeast,1725.55230
2,28,1,33.000,3,0,southeast,4449.46200
3,33,1,22.705,0,0,northwest,21984.47061
4,32,1,28.880,0,0,northwest,3866.85520


In [ ]:
# Заменяем регион на набор отдельных колонок (is_southwest, is_southeast и тп)
df = pd.get_dummies(df, columns=['region'])
df.head()

,age,sex,bmi,children,smoker,charges,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,16884.92400,0,0,0,1
1,18,1,33.770,1,0,1725.55230,0,0,1,0
2,28,1,33.000,3,0,4449.46200,0,0,1,0
3,33,1,22.705,0,0,21984.47061,0,1,0,0
4,32,1,28.880,0,0,3866.85520,0,1,0,0


## Формируем признаки и целевую переменную

In [ ]:
## Сформируйте признаки и целевую переменную

In [ ]:
x=df[['age','sex','bmi','children','smoker','region_northeast','region_northwest','region_southeast','region_southwest']]
y=df[['charges']]

In [ ]:
x

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,19,0,27.900,0,1,0,0,0,1
1,18,1,33.770,1,0,0,0,1,0
2,28,1,33.000,3,0,0,0,1,0
3,33,1,22.705,0,0,0,1,0,0
4,32,1,28.880,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
1333,50,1,30.970,3,0,0,1,0,0
1334,18,0,31.920,0,0,1,0,0,0
1335,18,0,36.850,0,0,0,0,1,0
1336,21,0,25.800,0,0,0,0,0,1


In [ ]:
y

,charges
0,16884.92400
1,1725.55230
2,4449.46200
3,21984.47061
4,3866.85520
...,...
1333,10600.54830
1334,2205.98080
1335,1629.83350
1336,2007.94500


## Разделяем данные на выборку для обучения/проверки

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [ ]:
X_train

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
430,19,1,33.100,0,0,0,0,0,1
1110,54,0,32.300,1,0,1,0,0,0
776,40,1,32.300,2,0,0,1,0,0
986,43,1,30.115,3,0,0,1,0,0
829,39,1,21.850,1,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...
88,46,0,27.740,0,0,0,1,0,0
927,57,0,20.100,1,0,0,0,0,1
442,18,1,43.010,0,0,0,0,1,0
40,24,0,26.600,0,0,1,0,0,0


In [ ]:
X_test

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
267,59,0,32.395,3,0,1,0,0,0
896,43,0,20.045,2,1,1,0,0,0
1102,29,1,38.940,1,0,0,0,1,0
991,38,0,27.835,2,0,1,0,0,0
983,27,0,30.590,1,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...
196,39,0,32.800,0,0,0,0,0,1
732,24,0,30.100,3,0,0,0,0,1
1111,38,1,38.390,3,1,0,0,1,0
241,33,0,22.135,1,0,1,0,0,0


In [ ]:
y_train

,charges
430,23082.95533
1110,11512.40500
776,6986.69700
986,8410.04685
829,6117.49450
...,...
88,8026.66660
927,12032.32600
442,1149.39590
40,3046.06200


In [ ]:
y_test

,charges
267,14590.63205
896,19798.05455
1102,3471.40960
991,7144.86265
983,16796.41194
...,...
196,5649.71500
732,4234.92700
1111,41949.24410
241,5354.07465


## Создаем и обучаем модель линейной регресии

In [ ]:
# Создайте и обучите модель

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
lr = LinearRegression()

In [ ]:
lr.fit(X_train, y_train)

LinearRegression()

In [ ]:
lr.coef_

array([[  255.29064899,  -237.40346247,   332.31404489,   462.3179212 ,
        23823.99712822,   555.48943197,   194.81361591,  -416.48166325,
         -333.82138463]])

## Получаем предсказание и оцениваем качество

In [ ]:
# Получите предсказание

In [ ]:
lr.predict(X_test)

array([[15611.68869179],
       [30784.63906065],
       [ 7993.95424586],
       [ 8272.91509716],
       [ 5917.92523077],
       [10174.08466025],
       [12875.70945251],
       [ 3591.4487571 ],
       [ 7625.56322655],
       [10814.24478157],
       [ 6248.08909281],
       [ 1977.39191248],
       [10744.77195861],
       [ 7268.12413024],
       [28638.41499618],
       [12254.35836028],
       [ 8963.51645856],
       [12259.20927918],
       [ 7196.56125013],
       [32106.87821278],
       [15903.40009394],
       [11272.36444539],
       [23249.44417982],
       [11761.30572326],
       [ 7557.81121489],
       [ 6622.3665655 ],
       [ 5621.58272724],
       [38450.92120384],
       [13949.86592871],
       [ 8918.61445532],
       [40026.17134892],
       [ 7208.27036507],
       [ 4298.98476645],
       [ 5596.01893969],
       [26687.18442425],
       [12117.70760539],
       [ 7510.91993603],
       [ 5467.72885306],
       [  425.00608378],
       [ 4020.0143677 ],


In [ ]:
# Оцените качество, при помощи метода mean_squared_error для тестовой выборки

In [ ]:
from sklearn.metrics import mean_squared_error

In [ ]:
mean_squared_error(y_test, lr.predict(X_test))

35473715.0660278

In [ ]:
mean_squared_error(y_train, lr.predict(X_train))

36768149.534275465

## Делаем предсказание для одного человека

In [ ]:
# Заполняем данные по конкретному человеку
data = [{
    "age": 20,
    "sex": 1,
    "bmi": 30,
    "children": 2,
    "smoker": 1,
    "region_northeast": 0,
    "region_northwest": 0,
    "region_southeast": 1,
    "region_southwest": 0
}]

In [ ]:
df_person = pd.DataFrame(data)
df_person.head()

,age,sex,bmi,children,smoker,region_northeast,region_northwest,region_southeast,region_southwest
0,20,1,30,2,1,0,0,1,0


In [ ]:
lr.predict(df_person)

array([[27011.7658931]])